# Advent of Code

## 2021-012-016
## 2021 016

https://adventofcode.com/2021/day/16

In [2]:
# Read the input file
with open('input.txt', 'r') as file:
    hex_data = file.read().strip()

# Convert hex to binary
binary_data = bin(int(hex_data, 16))[2:].zfill(len(hex_data) * 4)

# Function to parse a packet and calculate version sum
def parse_packet(binary, index=0):
    version_sum = 0
    version = int(binary[index:index + 3], 2)
    type_id = int(binary[index + 3:index + 6], 2)
    version_sum += version
    index += 6

    if type_id == 4:  # Literal value
        while binary[index] == '1':
            index += 5
        index += 5
    else:  # Operator
        length_type_id = int(binary[index], 2)
        index += 1
        if length_type_id == 0:
            total_length = int(binary[index:index + 15], 2)
            index += 15
            end = index + total_length
            while index < end:
                sub_version_sum, index = parse_packet(binary, index)
                version_sum += sub_version_sum
        else:
            num_sub_packets = int(binary[index:index + 11], 2)
            index += 11
            for _ in range(num_sub_packets):
                sub_version_sum, index = parse_packet(binary, index)
                version_sum += sub_version_sum

    return version_sum, index

# Parse the binary data and calculate the version sum
version_sum, _ = parse_packet(binary_data)
version_sum

965

In [3]:
# Function to parse a packet and calculate its value based on type ID
def parse_packet_with_value(binary, index=0):
    type_id_operations = {
        0: sum,
        1: lambda values: eval('*'.join(map(str, values))),
        2: min,
        3: max,
        5: lambda values: int(values[0] > values[1]),
        6: lambda values: int(values[0] < values[1]),
        7: lambda values: int(values[0] == values[1]),
    }

    type_id = int(binary[index + 3:index + 6], 2)
    index += 6

    if type_id == 4:  # Literal value
        value = 0
        while binary[index] == '1':
            value = (value << 4) | int(binary[index + 1:index + 5], 2)
            index += 5
        value = (value << 4) | int(binary[index + 1:index + 5], 2)
        index += 5
    else:  # Operator packet
        values = []
        length_type_id = int(binary[index], 2)
        index += 1
        if length_type_id == 0:
            total_length = int(binary[index:index + 15], 2)
            index += 15
            end = index + total_length
            while index < end:
                sub_value, index = parse_packet_with_value(binary, index)
                values.append(sub_value)
        else:
            num_sub_packets = int(binary[index:index + 11], 2)
            index += 11
            for _ in range(num_sub_packets):
                sub_value, index = parse_packet_with_value(binary, index)
                values.append(sub_value)

        # Compute the value based on the operator
        value = type_id_operations[type_id](values)

    return value, index

# Parse the binary data and calculate the value of the outermost packet
packet_value, _ = parse_packet_with_value(binary_data)
packet_value

116672213160